In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [8]:
df = pd.read_csv("LCs_CC.tar.gz")
df2 = pd.read_csv("LCs_Ia.tar.gz")
df3 = pd.read_csv("LCs_SESNe.tar.gz")

In [9]:
def make_explosion_array_for_single_light_curve(df):
    light_curve = df.to_numpy()
    
    g_bands = light_curve[:, [0, 4]]
    r_bands = light_curve[:, [5]]
    i_bands = light_curve[:, [6]]
    z_bands = light_curve[:, [7]]
    
    color_bands = [g_bands, r_bands, i_bands, z_bands]
    mean_color = []
    std_color = []
    explosion_row_num = 0
    
    mean, std = np.mean(g_bands[:45, [1]]), np.std(g_bands[:45, [1]])
    for row_num in range(len(g_bands)):
        if g_bands[row_num][1] < mean - (5*std):
                explosion_row_num = row_num
                break
    
    explosion_array = np.concatenate((g_bands[explosion_row_num:explosion_row_num+6, [0]] - g_bands[explosion_row_num, [0]],
                                     g_bands[explosion_row_num:explosion_row_num+6, [1]],
                                     r_bands[explosion_row_num:explosion_row_num+6],
                                     i_bands[explosion_row_num:explosion_row_num+6],
                                     z_bands[explosion_row_num:explosion_row_num+6],
                                     g_bands[explosion_row_num:explosion_row_num+6, [1]] - r_bands[explosion_row_num:explosion_row_num+6],
                                     r_bands[explosion_row_num:explosion_row_num+6] - i_bands[explosion_row_num:explosion_row_num+6],
                                     i_bands[explosion_row_num:explosion_row_num+6] - z_bands[explosion_row_num:explosion_row_num+6],
                                     light_curve[explosion_row_num:explosion_row_num+6, [8]]), axis=1)
    return explosion_array

In [23]:
start_row, end_row = 0, 290

final_array = make_explosion_array_for_single_light_curve(df[:290])

for row in df.iterrows():
    
    
    if (row[1][0] == 58750.0):
        if (row[0] == 0) or (row[0] == 290): #prevent passing empty dataframe or repeating first
            continue
        
        start_row = end_row
        end_row = row[0]
        sub_df = df.iloc[start_row:end_row, :]
        temp_explosion_array = make_explosion_array_for_single_light_curve(sub_df)

        final_array = np.append(final_array, temp_explosion_array, axis=0)

print(final_array)
print(np.shape(final_array))
new_df = pd.DataFrame(final_array, columns = ["Days after explosion", "g mag", "r mag", "i mag", "z mag", "g-r color", "r-i color", "i-z color", "Type"])
#new_df.to_csv('LCs_CC_modified.csv')

[[0.0 19.423363 19.41415 ... -0.012245000000000061 0.2148833880615193
  'SNII']
 [1.0 18.62696 18.648115 ... -0.034720000000000084 0.21824683471679762
  'SNII']
 [2.0 18.298187 18.445728 ... -0.10507000000000133 0.2392078052978519
  'SNII']
 ...
 [3.0 19.225338 19.008438 ... -0.1125179999999979 -0.06179851660155933
  'IIb']
 [4.0 18.85358 18.63475 ... -0.11749299999999963 -0.05597309497070313
  'IIb']
 [5.0 18.590357 18.381886 ... -0.2122489999999999 -0.11539106201171734
  'IIb']]
(50952, 9)
